In [ ]:
!pip install transformers==3.1.0

In [9]:
!pip install spacy
!pip install jieba

In [10]:
import torch
import transformers
import itertools

import spacy
import jieba

In [4]:
"""Load the multilingual BERT model and its tokenizer."""

model = transformers.BertModel.from_pretrained('bert-base-multilingual-cased')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-cased')

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [11]:
"""Input *tokenized* source and target sentences."""

src = 'Foodborne botulism is a serious, potentially fatal disease. It is an intoxication caused by ingestion of potent neurotoxins—the botulinum toxins—formed in contaminated foods. Person-to-person transmission of botulism does not occur. The manifestations, or symptoms, of intoxication can vary, and  is characterized by descending, flaccid paralysis that can cause respiratory failure. Early symptoms include marked fatigue, weakness and vertigo, usually followed by blurred vision, dry mouth and difficulty in swallowing and speaking. Vomiting, diarrhoea, constipation and abdominal swelling may also occur. The disease can progress to weakness in the neck and arms, after which the respiratory muscles and muscles of the lower body are affected. There is no fever and no loss of consciousness. Symptoms usually appear from several hours up to 8 days following consumption of contaminated food.'
tgt = '食源性肉毒中毒是一种严重且可能致命的疾病。这是一种因摄入受污染食物中形成的强效神经毒素（肉毒杆菌毒素）而引起的中毒。肉毒中毒不会发生人际传播。中毒表现或症状可能不尽相同，其特征是下行性弛缓性麻痹，可导致呼吸衰竭。早期症状包括明显疲劳、虚弱和眩晕，往往随后会出现视力模糊、口干以及吞咽和说话困难。也可能出现呕吐、腹泻、便秘和腹部肿胀。该病可发展为颈部和手臂无力，之后会累及呼吸肌和下半身肌肉。没有发热，也不会失去意识。症状通常在食用受污染的食物后数小时至8天内出现。'

In [14]:
# Load spaCy English model
nlp_en = spacy.load("en_core_web_sm")

# Tokenize English sentence
sent_src = [token.text for token in nlp_en(src)]

# Tokenize Chinese sentence
sent_tgt = list(jieba.cut(tgt))

# Print tokenized sentences
print("English tokens:", sent_src)
print("Chinese tokens:", sent_tgt)

English tokens: ['Foodborne', 'botulism', 'is', 'a', 'serious', ',', 'potentially', 'fatal', 'disease', '.', 'It', 'is', 'an', 'intoxication', 'caused', 'by', 'ingestion', 'of', 'potent', 'neurotoxins', '—', 'the', 'botulinum', 'toxins', '—', 'formed', 'in', 'contaminated', 'foods', '.', 'Person', '-', 'to', '-', 'person', 'transmission', 'of', 'botulism', 'does', 'not', 'occur', '.', 'The', 'manifestations', ',', 'or', 'symptoms', ',', 'of', 'intoxication', 'can', 'vary', ',', 'and', ' ', 'is', 'characterized', 'by', 'descending', ',', 'flaccid', 'paralysis', 'that', 'can', 'cause', 'respiratory', 'failure', '.', 'Early', 'symptoms', 'include', 'marked', 'fatigue', ',', 'weakness', 'and', 'vertigo', ',', 'usually', 'followed', 'by', 'blurred', 'vision', ',', 'dry', 'mouth', 'and', 'difficulty', 'in', 'swallowing', 'and', 'speaking', '.', 'Vomiting', ',', 'diarrhoea', ',', 'constipation', 'and', 'abdominal', 'swelling', 'may', 'also', 'occur', '.', 'The', 'disease', 'can', 'progress', 

In [16]:
"""Run the model and print the resulting alignments."""

# pre-processing
token_src, token_tgt = [tokenizer.tokenize(word) for word in sent_src], [tokenizer.tokenize(word) for word in sent_tgt]
wid_src, wid_tgt = [tokenizer.convert_tokens_to_ids(x) for x in token_src], [tokenizer.convert_tokens_to_ids(x) for x in token_tgt]
ids_src, ids_tgt = tokenizer.prepare_for_model(list(itertools.chain(*wid_src)), return_tensors='pt', model_max_length=tokenizer.model_max_length, truncation=True)['input_ids'], tokenizer.prepare_for_model(list(itertools.chain(*wid_tgt)), return_tensors='pt', truncation=True, model_max_length=tokenizer.model_max_length)['input_ids']
sub2word_map_src = []
for i, word_list in enumerate(token_src):
  sub2word_map_src += [i for x in word_list]
sub2word_map_tgt = []
for i, word_list in enumerate(token_tgt):
  sub2word_map_tgt += [i for x in word_list]

# alignment
align_layer = 8
threshold = 1e-3
model.eval()
with torch.no_grad():
  out_src = model(ids_src.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]
  out_tgt = model(ids_tgt.unsqueeze(0), output_hidden_states=True)[2][align_layer][0, 1:-1]

  dot_prod = torch.matmul(out_src, out_tgt.transpose(-1, -2))

  softmax_srctgt = torch.nn.Softmax(dim=-1)(dot_prod)
  softmax_tgtsrc = torch.nn.Softmax(dim=-2)(dot_prod)

  softmax_inter = (softmax_srctgt > threshold)*(softmax_tgtsrc > threshold)

align_subwords = torch.nonzero(softmax_inter, as_tuple=False)
align_words = set()
for i, j in align_subwords:
  align_words.add( (sub2word_map_src[i], sub2word_map_tgt[j]) )

# printing
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

for i, j in sorted(align_words):
  print(f'{color.BOLD}{color.GREEN}{sent_src[i]}{color.END}==={color.BOLD}{color.YELLOW}{sent_tgt[j]}{color.END}')

Foodborne===食源性
botulism===肉毒
botulism===中毒
is===是
a===一种
serious===严重
,===且
potentially===可能
fatal===致命
disease===疾病
.===。
It===这是
is===这是
an===一种
intoxication===中毒
caused===引起
by===因
ingestion===摄入
of===的
potent===强效
neurotoxins===神经毒素
botulinum===肉毒
botulinum===杆菌
toxins===神经毒素
toxins===毒素
formed===形成
in===摄入
contaminated===污染
foods===食物
.===。
Person===人际
transmission===传播
of===的
botulism===肉毒
botulism===中毒
not===不会
occur===发生
.===。
manifestations===表现
or===或
symptoms===症状
intoxication===中毒
can===可能
vary===不尽相同
and===，
is===其
characterized===特征
descending===性
flaccid===下行
paralysis===麻痹
that===，
can===可
cause===导致
respiratory===呼吸衰竭
failure===呼吸衰竭
.===。
Early===早期
symptoms===症状
include===包括
marked===明显
fatigue===疲劳
fatigue===眩晕
,===、
weakness===虚弱
and===和
,===，
,===，
usually===往往
by===出现
blurred===弛缓
blurred===眩晕
vision===视力
mouth===干
difficulty===困难
swallowing===吞咽
speaking===说话
.===。
Vomiting===呕吐
,===、
,===、
and===和
abdominal===腹部
swelling===腹泻
swelling===肿胀
may===会
also===也
occu